In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.multiprocessing as mp
from FPN import Features, FPNetwork , classificationhead , bboxhead
from Loss import Lossfunction
from datasets import load_dataset
import torch.optim as optim
from torch.cuda.amp import  GradScaler
from torch.amp import autocast
import random
import gc
from dataset_convert import AnchorGenerator, FaceDetectionDataset
# device = torch.device("mps")
scaler = GradScaler()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mp.set_start_method('spawn', force=True)

print(device)

/home/faces2.0/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


/tmp/ipykernel_528/371093317.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [2]:
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)

resnet_backbone = nn.Sequential(*list(resnet50.children())[:-2]).to(device)
layers_r50= {'4.0':256,  #-> 160,160 
     '4.1':512,  #->  80,80
     '4.2':1024, #-> 40,40
     '4.3':2048  #-> 20,20
     }

Using cache found in /home/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


In [3]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model = model.features.to(device)

Using cache found in /home/.cache/torch/hub/pytorch_vision_v0.10.0
/home/faces2.0/.conda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/faces2.0/.conda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
dataset = load_dataset("CUHK-CSE/wider_face")
train_dataset = dataset['train'].with_format("torch")
val_dataset = dataset['validation'].with_format("torch")

In [5]:
extractor = Features(resnet_backbone,layers=list(layers_r50.keys()))
topdown = FPNetwork(in_channels=layers_r50, out_channels=256)
classifier = classificationhead(channels=256, num_anchors= 12, num_of_classes= 1)
bboxregression = bboxhead(channels= 256 , num_anchors= 12)
loss =Lossfunction(lambd=10)
anchors = AnchorGenerator()
data = FaceDetectionDataset(train_dataset,anchors)
vali_data = FaceDetectionDataset(val_dataset,anchors)

In [6]:
# sd_extractor = torch.load('/home/faces2.0/models/extractor.pt',map_location=torch.device('cpu'))
# sd_classifier = torch.load('/home/faces2.0/models/classifier.pt',map_location=torch.device('cpu'))
# sd_bboxregression = torch.load('/home/faces2.0/models/bboxregression.pt',map_location=torch.device('cpu'))
# sd_topdown = torch.load('/home/faces2.0/models/topdown.pt',map_location=torch.device('cpu'))

# extractor.load_state_dict(sd_extractor)
# topdown.load_state_dict(sd_topdown)
# classifier.load_state_dict(sd_classifier)
# bboxregression.load_state_dict(sd_bboxregression)


In [7]:
# extractor = Features(model,['3','6', '13','18'])
# topdown = FPNetwork(out_channels=256)
# classifier = classificationhead(channels=256, num_anchors= 12, num_of_classes= 1)
# bboxregression = bboxhead(channels= 256 , num_anchors= 12)
# loss =Lossfunction(lambd=10)
# anchors = AnchorGenerator()
# data = FaceDetectionDataset(train_dataset,anchors)
# vali_data = FaceDetectionDataset(val_dataset,anchors)

In [8]:
torch.cuda.empty_cache()
print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

Allocated: 0.13 GB
Cached: 0.14 GB


In [9]:
def forward(p):
    features = extractor.extract(p)
    newfeatures = topdown(features)
    output = {}
    for key in list(newfeatures.keys()):
        temp = {}
        temp["bbox"] = bboxregression(newfeatures[key])
        temp["cls"] = classifier(newfeatures[key])
        output[key] = temp
    return output

In [10]:
kk =data[10310][0].unsqueeze(0).cuda()
jk = forward(kk)
ll = loss(jk, data[10310][1])
print(ll)

tensor(99367.1953, device='cuda:0', grad_fn=<AddBackward0>)


In [11]:
torch.sigmoid(jk['4.3']['cls'])

tensor([[[0.5340],
         [0.4859],
         [0.3539],
         ...,
         [0.2979],
         [0.6345],
         [0.3625]]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [12]:
def savea():
    torch.save(extractor.state_dict(),'/home/faces2.0/models/extractor.pt')
    torch.save(topdown.state_dict(),'/home/faces2.0/models/topdown.pt')
    torch.save(classifier.state_dict(),'/home/faces2.0/models/classifier.pt')
    torch.save(bboxregression.state_dict(),'/home/faces2.0/models/bboxregression.pt')

In [13]:
total_params=sum(p.numel() for p in extractor.parameters())
total_params+= sum(p.numel() for p in topdown.parameters())
total_params+= sum(p.numel() for p in classifier.parameters())
total_params+= sum(p.numel() for p in bboxregression.parameters())

print(total_params)

31719572


In [16]:
learning_rate = 1e-3

def train_with_accumulation(epochs: int, training_data, validation_data=None, accumulation_steps=4):
    # Set models to training mode
    extractor.train()
    topdown.train()
    classifier.train()
    bboxregression.train()
    
    # Enable gradient checkpointing
    if hasattr(extractor, 'gradient_checkpointing_enable'):
        extractor.gradient_checkpointing_enable()
    if hasattr(topdown, 'gradient_checkpointing_enable'):
        topdown.gradient_checkpointing_enable()
    
    optimizer = optim.Adam(
        list(extractor.parameters()) +
        list(topdown.parameters()) +
        list(classifier.parameters()) +
        list(bboxregression.parameters()), 
        lr=learning_rate
    )
    
    loss_history = {}
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        batch_count = 0
        running_loss = 0.0
        
        print(f"\nEpoch {epoch + 1}/{epochs}")
        print("-" * 30)
        try:
            for batch_idx, (image, bbox) in enumerate(training_data):
                # Mixed precision forward pass
                optimizer.zero_grad()
                
                with autocast('cuda'):
                    model_pred = forward(image.cuda())
                    ll = loss(model_pred, bbox) / accumulation_steps  # Scale loss
                # del image , model_pred , bbox
                # Backward pass
                scaler.scale(ll).backward()
                
                # Step optimizer every accumulation_steps
                if (batch_idx + 1) % accumulation_steps == 0:
                    scaler.step(optimizer)
                    scaler.update()
                    # for name, param in classifier.named_parameters():
                    # # print(param.grad)
                    #     if param.grad is not None:
                    #         print(f"Gradients for {name}:")
                    #         print(param.grad)

                    #     else :
                    #         print("swahaa")
                            
                    optimizer.zero_grad()
                
                # Accumulate loss
                batch_loss = ll.item() * accumulation_steps  # Unscale for logging
                epoch_loss += batch_loss
                running_loss += batch_loss
                batch_count += 1
                
                # Print progress
                if (batch_idx + 1) % 10 == 0:
                    avg_running_loss = running_loss / 10
                    # print("-" * 30)
                    print(f"Batch {batch_idx + 1}: Avg Loss = {avg_running_loss:.6f}")
                    # print("-" * 30)
                    running_loss = 0.0

                # Memory 
                del ll
                torch.cuda.empty_cache()
                gc.collect()
                # if batch_idx == 0:
                #     # print(image.shape)
                #     break
                del image , model_pred , bbox

        except Exception as e :
            print (e)
            print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
            print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
            torch.cuda.empty_cache()
            print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
            print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
            break

        except KeyboardInterrupt:
            print("INTERRUPTED!!")
            return loss_history   

                
        if batch_count % accumulation_steps != 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        avg_epoch_loss = epoch_loss / batch_count if batch_count > 0 else 0
        loss_history[epoch] = avg_epoch_loss
        print("-" * 30)
        print(f"Epoch {epoch + 1} Average Loss: {avg_epoch_loss:.6f}")
        print("-" * 30)
        
        
        # Cleanup at epoch end
        gc.collect()
        torch.cuda.empty_cache()
    # save()
    return loss_history


In [17]:
torch.cuda.empty_cache()
batch_size = 1
epochs = 5
training_data = DataLoader(
    data,
    batch_size=8, 
    num_workers=4,     # Reduce if GPU memory is full
    pin_memory=True,         # Faster CPU->GPU transfer
    persistent_workers=True, # Reuse workers across epochs
    prefetch_factor=4,       # Prefetch more batches
    drop_last=True,
    shuffle=True
)
validation_data = DataLoader(vali_data,
    batch_size=batch_size, 
    num_workers=4,     # Reduce if GPU memory is full
    pin_memory=True,         # Faster CPU->GPU transfer
    persistent_workers=True, # Reuse workers across epochs
    prefetch_factor=4,       # Prefetch more batches
    drop_last=True,
    shuffle=True
)

In [18]:
lossdata = train_with_accumulation(epochs = 1, training_data = training_data)


Epoch 1/1
------------------------------
Batch 10: Avg Loss = 2752.284961
Batch 20: Avg Loss = 2905.721875
Batch 30: Avg Loss = 47419.616541
Batch 40: Avg Loss = 2749.637842
Batch 50: Avg Loss = 1941.398328
Batch 60: Avg Loss = 1926.071149
Batch 70: Avg Loss = 1444.090002
Batch 80: Avg Loss = 1710.181116
Batch 90: Avg Loss = 5421.949655
Batch 100: Avg Loss = 1838.414673
Batch 110: Avg Loss = 1485.278040
Batch 120: Avg Loss = 1372.068622
Batch 130: Avg Loss = 2097.322662
Batch 140: Avg Loss = 1508.403513
Batch 150: Avg Loss = 2198.986566
Batch 160: Avg Loss = 2093.737555
Batch 170: Avg Loss = 1366.074707
Batch 180: Avg Loss = 1745.182330
Batch 190: Avg Loss = 1678.669873
Batch 200: Avg Loss = 12291.848309
Batch 210: Avg Loss = 1444.694391
Batch 220: Avg Loss = 4356.833752
Batch 230: Avg Loss = 32713.788766
Batch 240: Avg Loss = 34598.653943
Batch 250: Avg Loss = 1567.134772
Batch 260: Avg Loss = 2370.392535
INTERRUPTED!!


In [ ]:
loss = Lossfunction()
extractor.eval()
topdown.eval()
classifier.eval()
bboxregression.eval()
accumulation_steps = 4
for batch_idx, (image, bbox) in enumerate(validation_data):

    with autocast('cuda'):
        model_pred = forward(image.cuda())
        ll = loss(model_pred, bbox) / accumulation_steps  # Scale loss
    del image , model_pred , bbox
    print(ll)

In [28]:
torch.cuda.empty_cache()
print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

Allocated: 4.44 GB
Cached: 4.80 GB


In [20]:
data[1000][0].unsqueeze(0).shape

torch.Size([1, 3, 640, 640])

In [21]:
def check(i, threshold , layers = ['4.0','4.1','4.2','4.3']):
    s=forward(vali_data[i][0].unsqueeze(0).cuda())
    t= vali_data[i][1]
    ls = loss(s,t)
    print(f"Loss:{ls}")
    print(torch.sigmoid(s['4.1']['cls']))
    for key,layer in enumerate(layers):
        print(f"prediction: {(torch.sigmoid(s[layer]['cls']).cpu()>threshold).sum()} out of total {s[layer]['cls'].cpu().shape} ")
        print(f"truth: {t[len(layers)-key-1]['cls_targets'].sum()} out of total {t[len(layers)-key-1]['cls_targets'].shape}")

In [74]:
data

In [25]:
check(1109, 0.5)

Loss:53560.125
tensor([[[0.5186],
         [0.5069],
         [0.5119],
         ...,
         [0.5304],
         [0.5226],
         [0.5248]]], device='cuda:0', grad_fn=<SigmoidBackward0>)
prediction: 187398 out of total torch.Size([1, 307200, 1]) 
truth: 0 out of total torch.Size([1, 307200, 1])
prediction: 51337 out of total torch.Size([1, 76800, 1]) 
truth: 0 out of total torch.Size([1, 76800, 1])
prediction: 12921 out of total torch.Size([1, 19200, 1]) 
truth: 0 out of total torch.Size([1, 19200, 1])
prediction: 2927 out of total torch.Size([1, 4800, 1]) 
truth: 32 out of total torch.Size([1, 4800, 1])


In [ ]:
savea()